In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit, openmm
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)



In [2]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/24/"
phase = 'complex'
sim_number = 1
old_aa_name = 'asn'
new_aa_name = 'arg'


In [3]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

# Define simulation parameters
# nsteps_eq = 25000 # 100 ps 
nsteps_eq = 1
nsteps_neq = 10 
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
cache_length = 1 



In [4]:
# Read in vanilla htf
i = os.path.basename(os.path.dirname(outdir))
with open(os.path.join(outdir, f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)




In [5]:
# Read in lambda = 0 cache
with open(os.path.join(outdir, f"{i}_{phase}_{old_aa_name}_{cache_length}ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[sim_number - 1]
system = htf.hybrid_system

# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep)


shape: (100, 200492, 3)
array.shape: (60147600,)


In [6]:
# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)



In [7]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)


In [8]:
state = context.getState(getPositions=True, getVelocities=True)
with open(os.path.join(outdir, f"state.xml"), 'w') as outfile:
    state_xml = openmm.XmlSerializer.serialize(state)
    outfile.write(state_xml)

In [9]:
# Run eq forward (0 -> 1)
forward_eq_old, forward_eq_new = list(), list()
for fwd_step in range(nsteps_eq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds

# Run neq forward (0 -> 1)
forward_works_master = list()
forward_neq_old, forward_neq_new = list(), list()
forward_works = [integrator.get_protocol_work(dimensionless=True)]
for fwd_step in range(nsteps_neq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    print(f"fwd_step: {fwd_step}")
    forward_works.append(integrator.get_protocol_work(dimensionless=True))
forward_works_master.append(forward_works)



fwd_step: 0
fwd_step: 1
fwd_step: 2
fwd_step: 3
fwd_step: 4
fwd_step: 5
fwd_step: 6
fwd_step: 7
fwd_step: 8
fwd_step: 9


In [10]:
# Read in lambda = 1 cache, if necessary
with open(os.path.join(outdir, f"{i}_{phase}_{new_aa_name}_{cache_length}ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[sim_number - 1]
context.setPositions(positions)



shape: (100, 200492, 3)
array.shape: (60147600,)


In [ ]:
# Run eq reverse (1 -> 0)
reverse_eq_old, reverse_eq_new = list(), list()
for rev_step in range(nsteps_eq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    
# Run neq reverse (1 -> 0)
reverse_works_master = list()
reverse_neq_old, reverse_neq_new = list(), list()
reverse_works = [integrator.get_protocol_work(dimensionless=True)]
for rev_step in range(nsteps_neq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    reverse_works.append(integrator.get_protocol_work(dimensionless=True))
reverse_works_master.append(reverse_works)



In [ ]:
# Save works
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward.npy"), 'wb') as f:
    np.save(f, forward_works_master)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse.npy"), 'wb') as f:
    np.save(f, reverse_works_master)
